# A notebook for the Bayesian probabilistic framework



In this notebook we aim to use the __Bayesian probabilistic__ parameter estimation.

For this reason we will be using the _HEIGHT-WEIGHT_ data-set.

In [ ]:
# The below packages are required for data manipulation

import Plots, DataFrames, CSV

hw = CSV.read(
    "hw_data.csv",
    DataFrames.DataFrame,
    header=false
)

;

In [ ]:
# making sure that the data was read in
Plots.scatter(
    hw[!,1],hw[!,2],
    size=(360,180),label=false
)
# the default backend is the GR, 
# we can live with it.

## Help on using help in a _julialang_ notebook

You can use the help system in julialang using the inline help or the online documentation:
1. if online help is used -- e.g. in juliaplots.org -, you might want to explore the optional set of arguments,
2. you can prefix a command with ? and if it is only that inside the cell.

For example, we can check that
```julia
?range
```
will display the inline help in the system.

## Setting up the individual likelihood function




We are using the Gaussian likelihood

\begin{equation}
P(\ (y,x) |\ \theta,{\cal{F}})
=
\frac{1}{\sigma\sqrt{2\pi}}\exp\left(-\frac{\|y-\theta^T x\|^2}{2\sigma^2} \right)
\end{equation}


The above form of the likelihood function highlights that __data pair -- the input and output --__ is given a "likelihood".

The code is next.

In [ ]:
# the gauss pdf
function pdf_gauss(z::T;σ=1) where T <: Real
    exp(- z^2 / (2σ^2)) / (σ*sqrt(2π))
end

# likelihood for the linear model
lik_one( (x,y),θ; σ=1 ) = 
    pdf_gauss(y - (x'*θ)[1]; σ = σ)

lik_data(X,y,θ; σ=1) = prod([
    lik_one((X[ii,:], y[ii]),θ; σ = σ )
    for ii = 1:length(y)
])
;

## Checking the likelihood function

The likelihood function has to be such that the integral is one; we code this by using the _Riemann formula_.

$$
val = \int_{-\infty}^\infty \ \mathrm{pdf\_gauss}(z) dz \\
= \int_{-v}^v \ \mathrm{pdf\_gauss}(z) dz
$$

In [ ]:
# setting a lower and upper limit
v_lim = 20  #
n_div = 501

z_div = range(-v_lim,v_lim,length=n_div)
δ_z   = z_div[2] - z_div[1]

# we simulate the integration on [-inf,inf]
total = sum([pdf_gauss(z) for z in z_div])*δ_z

We see that the value we have is close to one, meaning that the function is a _probability density function_.

## Setting up the data-set

In [ ]:
n_data = 4  # this value should be 
            # smaller than 19 - the length of data


X = [ones(n_data,1) hw[1:n_data,1]]
y = Vector(hw[1:n_data,2])

;

In [ ]:
# plotting the restricted data-set
p_data = Plots.scatter(X[:,2],y,
    legend=false,
    title="Small data-set",
    size=(400,250),titlefontsize=9,
    xlim=(145,200),ylim=(45,115)
)

# Computing the Bayesian posterior distribution

The computation of the posterior requires that __we know__ the prior distribution for the parameters and that we know the __likelihood model__ of the data, that is:
$$
p_0(\theta) \propto \exp\left(-\frac{\|\theta\|^2}{2\sigma_p^2}\right) \\
P({\cal{D}}\ |\theta,{\cal{F}}) \propto \prod_{n=1}^N p( (y_n,x_n)\ | \theta,{\cal{F}})
$$
__Note__ that in the above formulation we are not that careful w.r.to the exact scaling of the functions - the reason is that in the Bayesian formalism the _FULL DATA PROBABILITY_ is not computable but one still will have to normalize.

According to Bayes' rule, the a-posteriori distribution is computed as:
$$
p_{post}(\theta |\ {\cal{D}},{\cal{F}}) = \frac{p({\cal{D}} |\ \theta,{\cal{F}})p_0(\theta|{\cal{F}})}{p({\cal{D}}|\ {\cal{F}})}
$$
where ${\cal{F}}$ is the model family - here the _linear models_.

Since both the likelihood _and_ the prior are given up to a scaling constant, and the denominator $p({\cal{D}}|\ {\cal{F}})$ is not easy to compute, we write the posterior as:
$$
p_{post}(\theta) \propto 
p({\cal{D}} |\ \theta,{\cal{F}})p_0(\theta|{\cal{F}})
.
$$

The above means that the posterior has a scaling $Z$, the true posterior is:
$$
p_{post}(\theta) = \frac{1}{Z}
p({\cal{D}} |\ \theta,{\cal{F}})p_0(\theta|{\cal{F}})
.
$$


## The computation of the posterior

There are two ways of computing the posterior: through sampling and through analytical consideration.

### Posterior computation through sampling

We use the Bayes rule to determine __moments__ of the posterior:
$$
\langle f(\theta) \rangle_p = \int_\Omega\ f(\theta) dp_{post}(\theta|{\cal{D},\cal{F}}) = \int_\Omega\ f(\theta) p({\cal{D}} |\ \theta,{\cal{F}})p_0(\theta|{\cal{F}}) d\theta
= \frac{1}{T}\sum_{t=1}^T f(\theta_t) p({\cal{D}} |\ \theta_t,{\cal{F}})
$$
and we see that it is a _weighted_ sum where the weights $w_t=p({\cal{D}} |\ \theta_t,{\cal{F}})$.

In practice the calculation uses the fact that
$$
1 = \langle 1 \rangle_p
= \frac{1}{Z}\frac{1}{T}\sum_{t=1}^T 1 p({\cal{D}} |\ \theta_t,{\cal{F}})
$$
giving the possibility to express $Z$.


Below we have the code for sampling.

We sample a multidimensional Gaussian with variance $\sigma_0$ and - optionally with mean $\mu_0$:

```julia
sample_theta(dim, variance; mean=0)
```

In [ ]:
sample_theta(dim,σ_0,μ_0=zeros(dim))= μ_0 + σ_0 * randn(dim,1)

In [ ]:
# sampling from the distribution
n_sample = 50_000
θ_mean = [-100, 1]
θ_std  = 50
θ_dim  = size(X,2)

# the sampling code
θ_sam  = [
    sample_theta(θ_dim,θ_std,θ_mean) 
    for _ = 1:n_sample
]
;

In [ ]:
th = hcat(θ_sam...)'
p_sam = Plots.scatter(
    th[:,1],th[:,2],
    size=(400,250),
    markerstrokewidth=0,markersize=1,
    legend=false)

In [ ]:
# computing weights
w = [lik_data(X,y,θ; σ=14) for θ in θ_sam]
# denominator
Z = sum(w)
# numerator
num = sum(w.*θ_sam)

# the empirical mean
num ./ Z

In [ ]:
Plots.histogram(log10.(w),
    bins=3000,
    size=(400,140),
    legend=false
)

## Analyzing the algorithm to compute the mean

The most likely parameters using a sample of $50.000$ data leads to a pair $\theta=[-77,0.9]$ but this is based on individual weights that are small - only $100$ of them are larger than $10^{-8}$.

This is done by counting "_the most significant_" ones:
```julia
count(w .> 1e-8)
```

The number - which in the particular setting was $100$ indicates that the results will be unstable - the denominator in the formula for the mean is __way too small__. 

Additionally, if there are more, than a few data items, the data likelihood term is becoming even smaller, reducing the confidence and increasing the uncertainty in the estimation.

In [ ]:
count(w .> 1e-8)

### Printing out a notebook

When printing the notebook, I suggest to use the __SAVE-TO-PDF__ function of your browser. This works best if you are using the ``GR`` backend for plotting.

# Your task

1. Interpret the result of the Bayesian scheme using the _two plot handles_ ``p_data`` and ``p_sam`` by 
  1. plotting the $100$ _likely_-est functions on ``p_data`` plot.
  2. plotting - with scatter - the _likely_ parameters on ``p_sam``, i.e in the parameter space.

  Make the plots visible - by using ``xlim``, ``lyim`` and other parameters - and explain your results in a notebook cell.
  
  (3p).

1. Change the data-size to $10$, re-do the above calculation _and_ select ``meaningful`` samples.

  You should see that the number of __good__ parameters diminished significantly. Explain your results.
  
  (3p).
  
1. (Using new sections)
 
  Re-do the calculation using the sampling - but store only the __logarithm__ of the data likelihood values and in computing the samples, use the "_broadcast_" version of the ``exp.(w)`` function.

  (2p).

  1. prove that you can add __any constant__ to each $lw = \log(w_t)$ and this does not change the output of the Bayesian calculation.
  
    (1p).
  
  1. For the sample - say of size $50.000$ - compute the maximum value of ``lw`` and subtract this from each log-weight - making the maximum weights equal to $\exp(0)=1$.
  
    (1p).

  1. with the transformed weights compute the mean of the parameters.
     
    (1p).

  1. plot the _likeliest_ parameters as from above - using the new computation of weights.
  
1. Compute - using the argumentation from above - the sample covariance of the parameter vector.
  
  (4p).

